In [1]:
from datasets import Dataset
from app import retrieve_relevant_documents, emb_text, model, embedding_model
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnableLambda
from langchain_core.documents import Document

f:\Dissertation\prod-rag-chat\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2025-08-16 16:34:21 - Loaded .env file


In [ ]:
def setup_standalone_rag_chain():
    """Setup a standalone RAG chain for testing without Chainlit session."""
    
    def get_context_and_history(inputs):
        """Retrieve context without session history."""
        query = inputs["question"]
        relevant_docs = retrieve_relevant_documents(query, limit=5)
        print("Relevant documents:", relevant_docs[0] if relevant_docs else "No documents found")
        
        # Convert dictionaries to Document objects for LangChain
        doc_objects = []
        for doc in relevant_docs:
            doc_obj = Document(
                page_content=doc.get('text', ''),
                metadata=doc.get('metadata', {})
            )
            doc_objects.append(doc_obj)

        return {
            "question": query,
            "context": doc_objects,
            "history": []  # Empty history for testing
        }
    
    system_prompt = """You are a helpful assistant specialising in developing non-discriminatory competence standards and disability support, reasonable adjustments, and equality legislation.

When answering questions, you should:
1. Use the provided context documents to inform your response
2. Be accurate and helpful
3. If the context doesn't contain relevant information, say so clearly
4. Always reply in English
5. Provide clear recommendations and examples wherever applicable
6. Do not make assumptions about the user's knowledge or background
7. If the user asks for a specific law or regulation, provide a brief explanation and cite relevant documents if available.
8. Do not overemphasize disability in your responses, but rather focus on the support and adjustments that can be made to ensure equality and inclusivity.
9. If the user query explicitly asks for a disability-related topic, provide a well-informed response based on the context documents.

Context documents:
{context} 

Please provide a clear response using the above context
"""

    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ])

    question_answer_chain = create_stuff_documents_chain(model, prompt)
    
    # Use a custom chain that properly handles our context and history
    def process_input_and_format(inputs):
        context_data = get_context_and_history(inputs)
        return {
            "context": context_data["context"],
            "question": context_data["question"],
            "history": context_data["history"]
        }
    
    chain = RunnableLambda(process_input_and_format) | question_answer_chain
    
    return chain

In [3]:

# Setup the RAG chain
rag_chain = setup_standalone_rag_chain()

questions = ["What are Provisions, Criteria and Practices?", 
             "What is 'reasonable'?",
             "What is 'substantial disadvantage'?",
            ]
ground_truths = [
    """The Equality and Human Rights Commission (EHRC) interprets PCPs as including:3 
+ arrangements (for example, for deciding who to admit)  
+ the way that education, or access to any benefit, service or facility is offered or provided  
+ one-off or discretionary decisions  
+ proposals or directions to do something in a particular way  
+ formal and informal policies  
+ rules""",

    """There are two key considerations of 'reasonableness' which can help when thinking through 
when an adjustment may be reasonable:4  
+ Could the adjustment be practicable in its application (is it possible)? 
+ Could the adjustment be effective in achieving its aim (will it work)? 
There is no need to prove that the adjustment is practicable and effective in advance, just 
that it might be. An adjustment should not be considered unreasonable if it does not remove 
the disadvantage fully; an adjustment which partially removes or reduces substantial 
disadvantage is also likely to be reasonable.""",

    """'Substantial' is defined in the Act as 'more than minor or trivial'. 
Examples of disadvantage recognised by the EHRC include: 
+ The additional time and effort expended by a disabled student 
+ The inconvenience, indignity, discomfort, or perceived disadvantage suffered by a 
disabled student 
+ The loss of opportunity or diminished progress experienced by a disabled student. """]





In [ ]:
answers = []
contexts = []

def clean_answer(answer):
    """Remove <think></think> tags and content from the answer."""
    import re
    # Remove everything between <think> and </think> tags, including the tags themselves
    cleaned = re.sub(r'<think>.*?</think>\s*', '', answer, flags=re.DOTALL)
    return cleaned.strip()

# Inference
for query in questions:
    # Get answer from the RAG chain
    answer = rag_chain.invoke({"question": query})
    # Clean the answer to remove thinking content
    cleaned_answer = clean_answer(answer)
    answers.append(cleaned_answer)
    
    # Get relevant documents for context
    relevant_docs = retrieve_relevant_documents(query, limit=5)
    context_texts = [doc['text'] for doc in relevant_docs]
    contexts.append(context_texts)

# To dict
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "reference": ground_truths
}

# Convert dict to dataset
dataset = Dataset.from_dict(data)



2025-08-16 16:34:53 - HTTP Request: POST https://api.studio.nebius.ai/v1/embeddings "HTTP/1.1 200 OK"
Relevant documents: {'text': 'What is a provision, criterion or practice? The phrase ‘provision, criterion or practice’ is not defined by the Act. These concepts should be construed widely so as to include, for example, any formal or informal policies, rules, practices, arrangements, criteria, procedures, activities or provisions. They can cover one-off decisions and actions. In simple terms, they are about the way an education provider does things. Example:', 'metadata': {'source': 'data\\technical-guidance-further-higher-education.docx', 'file_directory': 'data', 'filename': 'technical-guidance-further-higher-education.docx', 'last_modified': '2025-07-02T21:00:50', 'page_number': 95, 'languages': ['eng'], 'filetype': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document', 'category': 'CompositeElement', 'element_id': '3ae881ad6f81487213a9e234debf0921'}, 'score': 0.

In [5]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

result = evaluate(
    llm=model,
    embeddings=embedding_model,
    dataset = dataset, 
    metrics=[
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy,
    ],
)

df = result.to_pandas()

# evaluation_results = result.to_pandas()

# display_columns = ['user_input', 'answer_relevancy', 'faithfulness', 'context_precision', 'context_recall']
# formatted_results = evaluation_results[display_columns].to_markdown(index=False, numalign="left", stralign="left")

# print(formatted_results)


Evaluating:   0%|          | 0/12 [00:00<?, ?it/s]

2025-08-16 16:35:22 - HTTP Request: POST https://api.studio.nebius.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-16 16:35:22 - HTTP Request: POST https://api.studio.nebius.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-16 16:35:22 - HTTP Request: POST https://api.studio.nebius.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-16 16:35:22 - HTTP Request: POST https://api.studio.nebius.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-16 16:35:22 - HTTP Request: POST https://api.studio.nebius.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-16 16:35:22 - HTTP Request: POST https://api.studio.nebius.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-16 16:35:22 - HTTP Request: POST https://api.studio.nebius.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-16 16:35:22 - HTTP Request: POST https://api.studio.nebius.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-16 16:35:22 - HTTP Request: POST https://api.studio.nebius.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-16 16:35:22 - HTTP R

Evaluating:   8%|▊         | 1/12 [00:06<01:13,  6.67s/it]

2025-08-16 16:35:29 - HTTP Request: POST https://api.studio.nebius.ai/v1/embeddings "HTTP/1.1 200 OK"
2025-08-16 16:35:29 - HTTP Request: POST https://api.studio.nebius.ai/v1/embeddings "HTTP/1.1 200 OK"
2025-08-16 16:35:29 - HTTP Request: POST https://api.studio.nebius.ai/v1/embeddings "HTTP/1.1 200 OK"
2025-08-16 16:35:29 - HTTP Request: POST https://api.studio.nebius.ai/v1/embeddings "HTTP/1.1 200 OK"


Evaluating:  17%|█▋        | 2/12 [00:07<00:32,  3.29s/it]

...

Evaluating:  33%|███▎      | 4/12 [00:11<00:19,  2.46s/it]

2025-08-16 16:35:34 - HTTP Request: POST https://api.studio.nebius.ai/v1/chat/completions "HTTP/1.1 200 OK"


Evaluating:  42%|████▏     | 5/12 [00:12<00:13,  1.88s/it]

2025-08-16 16:35:35 - HTTP Request: POST https://api.studio.nebius.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-16 16:35:35 - HTTP Request: POST https://api.studio.nebius.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-16 16:35:35 - HTTP Request: POST https://api.studio.nebius.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-16 16:35:36 - HTTP Request: POST https://api.studio.nebius.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-16 16:35:38 - HTTP Request: POST https://api.studio.nebius.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-16 16:35:39 - HTTP Request: POST https://api.studio.nebius.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-16 16:35:39 - HTTP Request: POST https://api.studio.nebius.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-16 16:35:41 - HTTP Request: POST https://api.studio.nebius.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-16 16:35:43 - HTTP Request: POST https://api.studio.nebius.ai/v1/chat/completions "HTTP/1.1 200 OK"


Evaluating: 100%|██████████| 12/12 [00:42<00:00,  3.51s/it]


In [6]:
print(df)

                                     user_input  \
0  What are Provisions, Criteria and Practices?   
1                         What is 'reasonable'?   
2           What is 'substantial disadvantage'?   

                                  retrieved_contexts  \
0  [What is a provision, criterion or practice? T...   
1  [‘Reasonable’ means having regard to all of th...   
2  [The Act states that disadvantage must be subs...   

                                            response  \
0  **Provisions, Criteria, and Practices (PCPs)**...   
1  The term **"reasonable"** in the context of di...   
2  **Substantial disadvantage** refers to a situa...   

                                           reference  context_precision  \
0  The Equality and Human Rights Commission (EHRC...           0.500000   
1  There are two key considerations of 'reasonabl...           0.500000   
2  'Substantial' is defined in the Act as 'more t...           0.866667   

   context_recall  faithfulness  answer_rele